# Re-training of ActiNet model using existing old model

This notebook shows how to re-train the ActiNet model for deployment, given an existing model.

In [ ]:
import joblib
from datetime import datetime
import hashlib

from actinet.actinet import read
from actinet.accPlot import plotTimeSeries
from actinet.models import ActivityClassifier

WINSEC = 30 # seconds
SAMPLE_RATE = 100 # Hz
N_JOBS = 8 # Set to higher number for quicker execution, but don't exceed max.

existing_classifier_path = "src/actinet/ssl-ukb-c24-rw-20240308.joblib.lzma"

In [ ]:
classifier = ActivityClassifier(
    batch_size=512,
    device="cpu",
    window_sec=WINSEC,
    verbose=True,
    weights_path="models/c24_rw_20240308.pt"
)

print(classifier)

In [ ]:
classifier_file_name = f"models/ssl-ukb-c24-rw-{WINSEC}s-{datetime.now().strftime('%Y%m%d')}.joblib.lzma"

In [ ]:
old_classifier: ActivityClassifier = joblib.load(existing_classifier_path)

In [ ]:
classifier.labels = old_classifier.labels
classifier.hmms = old_classifier.hmms
classifier.model_weights = old_classifier.model_weights

In [ ]:
classifier.save(classifier_file_name)

We load the saved classifier to ensure the expected behaviour

In [ ]:
loaded_classifier: ActivityClassifier = joblib.load(classifier_file_name)

print(loaded_classifier)

In [ ]:
loaded_classifier.load_model()

In [ ]:
hashlib.md5(open(classifier_file_name,'rb').read()).hexdigest()

In [ ]:
data, info = read("data/sample.cwa.gz", resample_hz=None, verbose=True)

In [ ]:
y = loaded_classifier.predict_from_frame(data, 100, True, None, False)
p = plotTimeSeries(y, "Keep naps, no minimum sleep duration")

In [ ]:
y = loaded_classifier.predict_from_frame(data, 100, True,'5H', True)
p = plotTimeSeries(y, "Single sleep block per day, minimum 5 hour sleep")

In [ ]:
y = loaded_classifier.predict_from_frame(data, 100, True, '45min', False)
p = plotTimeSeries(y, "Keep naps, minimum 45 min sleep")